In [1]:
import os
# os.chdir(r'C:\Users\secn17444\OneDrive - WSP O365\pyproj\flow_env\src')
import pandas as pd
import openpyxl

In [2]:
# Path to your Excel file
# excel_file_path = r'c:\Users\secn17444\OneDrive - WSP O365\Projekt\Växjö\5142-10347420\PST\Arbetsmaterial\Flöden till Örsled PST och regn\Örsled_2021-10-19_-_2021-10_23_version_2_CNI_bara rådata.xlsm'
excel_file_path = r'c:\Users\chrini\Documents\Projekt\Örsled PST\Flöden till Örsled PST och regn\Örsled_2021-10-19_-_2021-10_23_version_2_CNI_bara rådata.xlsm'

# Read the Excel file
df = pd.read_excel(excel_file_path)

# Display the DataFrame
print(df.head())
# print(df['ASRL02 Örsled PST till Sundets ARV [m3/h]'])

             DateTime  ASRL02 Örsled PST till Sundets ARV [m3/h]  \
0 2021-10-19 00:00:00                                         39   
1 2021-10-19 00:01:00                                         38   
2 2021-10-19 00:02:00                                         32   
3 2021-10-19 00:03:00                                         32   
4 2021-10-19 00:04:00                                         35   

   AP247 Damsrydh [m³/h]  AP229 Stärkelsen  [m³/h]  AP220 Wessels  [m3/h]  \
0                    0.0                         0                  10.80   
1                    0.0                         0                  10.80   
2                    0.0                         0                   7.20   
3                    0.0                        29                   3.96   
4                    0.0                       188                   3.96   

   AP220 Wessels  [l/s]  
0                   3.0  
1                   3.0  
2                   2.0  
3                   1.1 

In [3]:
df.drop(columns=['AP220 Wessels  [l/s]'], inplace=True)

In [4]:
# Convert 'date' column to datetime
df['DateTime'] = pd.to_datetime(df['DateTime'])

# Set 'date' column as the index
df.set_index('DateTime', inplace=True)

In [5]:
df['Summaflöde till Örsled PST  [m3/h]'] = df['AP247 Damsrydh [m³/h]']+df['AP229 Stärkelsen  [m³/h]']+df['AP220 Wessels  [m3/h]']

In [6]:
# df['Kumsum-flöde till Örsled PST  [m3/h]'] = df['Summaflöde till Örsled PST  [m3/h]'].cumsum()
# df['Kumsum-flöde ASRL02 Örsled PST till Sundets ARV [m3/h]'] = df['ASRL02 Örsled PST till Sundets ARV [m3/h]'].cumsum()
df_ax2 = pd.DataFrame(index=df.index)
df_ax2['Pumpsumpkumsum [m3]'] = df['Summaflöde till Örsled PST  [m3/h]'].cumsum() - df['ASRL02 Örsled PST till Sundets ARV [m3/h]'].cumsum()

In [7]:
# # Indices of columns to drop
# columns_to_drop = [2, 3]  # Drop the second and fourth columns

# # Get the column names to drop
# columns_to_drop_names = [df.columns[i] for i in columns_to_drop]

# # Drop the columns
# df.drop(columns=columns_to_drop_names, inplace=True)

In [8]:
# df.drop(columns=['AP247 Damsrydh [m³/h]','AP229 Stärkelsen  [m³/h]','AP220 Wessels  [m3/h]'], inplace=True, errors='ignore')

In [9]:
print(df.index)

DatetimeIndex(['2021-10-19 00:00:00', '2021-10-19 00:01:00',
               '2021-10-19 00:02:00', '2021-10-19 00:03:00',
               '2021-10-19 00:04:00', '2021-10-19 00:05:00',
               '2021-10-19 00:06:00', '2021-10-19 00:07:00',
               '2021-10-19 00:08:00', '2021-10-19 00:09:00',
               ...
               '2021-10-23 22:51:00', '2021-10-23 22:52:00',
               '2021-10-23 22:53:00', '2021-10-23 22:54:00',
               '2021-10-23 22:55:00', '2021-10-23 22:56:00',
               '2021-10-23 22:57:00', '2021-10-23 22:58:00',
               '2021-10-23 22:59:00', '2021-10-23 23:00:00'],
              dtype='datetime64[ns]', name='DateTime', length=7141, freq=None)


In [10]:
print(df.columns[:])
print(df_ax2.columns[:])

Index(['ASRL02 Örsled PST till Sundets ARV [m3/h]', 'AP247 Damsrydh [m³/h]',
       'AP229 Stärkelsen  [m³/h]', 'AP220 Wessels  [m3/h]',
       'Summaflöde till Örsled PST  [m3/h]'],
      dtype='object')
Index(['Pumpsumpkumsum [m3]'], dtype='object')


In [11]:
# 2025-02-12: Den fungerande koden flyttades till src\Örsled_PST.py
from PyQt6.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QHBoxLayout, QCheckBox,QPushButton, QWidget, QLabel
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas, NavigationToolbar2QT
import matplotlib.dates as mdates
from matplotlib.dates import MO, WeekdayLocator
import sys
import numpy as np
import pandas as pd

import inspect
import re
def dprint(x): # https://stackoverflow.com/questions/32000934/print-a-variables-name-and-value/57225950#57225950
    frame = inspect.currentframe().f_back
    s = inspect.getframeinfo(frame).code_context[0]
    r = re.search(r"\((.*)\)", s).group(1)
    print("{} = {}".format(r,x))

class CustomNavigationToolbar(NavigationToolbar2QT):
    def __init__(self, canvas, parent=None, plot_window = None):
        super().__init__(canvas, parent)
        self.plot_window = plot_window
        self.addSeparator()
        self.addAction('Autoscale', self.autoscale)
        self.addSeparator()
        self.addAction('Autoscale LY', self.autoscaleLeftY)
        self.addSeparator()
        self.addAction('Reset X', self.plot_window.resetXAxis)
#         # Set fixed size for buttons
#         button_width = 100
#         button_height = 30

        # Add QLabel to display xAxisLen
        self.xAxisLenLabel = QLabel("xAxisLen: N/A")
        self.addWidget(self.xAxisLenLabel)

    def autoscale(self):
        ax1 = self.plot_window.ax1
        ax1.autoscale(axis='both')
        self.canvas.draw()
        self.update_xAxisLen()

    def autoscaleLeftY(self):
        series_visibleLeftY = self.plot_window.series_visible[:len(self.plot_window.df.columns)]
        df = self.plot_window.df
        selected_columns = df.loc[:, series_visibleLeftY]
        ylim = (selected_columns.min().min(), selected_columns.max().max())
        self.ax1.set_ylim(ylim)
        self.canvas.draw()

    def autoscaleRightY(self):
        series_visibleRightY = self.plot_window.series_visible[len(self.plot_window.df_ax2.columns):]
        df_ax2 = self.plot_window.df_ax2
        selected_columns = df_ax2.loc[:, series_visibleRightY]
        ylim = (selected_columns.min().min(), selected_columns.max().max())
        self.ax2.set_ylim(ylim)
        self.canvas.draw()

    def pan(self):
        super().pan()
        self.plot_window.resetXAxis()
        self.update_xAxisLen()

    def zoom(self):
        super().zoom()
        self.plot_window.resetXAxis()
        self.update_xAxisLen()

    def update_xAxisLen(self):
        ax1 = self.canvas.figure.gca()
        xlim = ax1.get_xlim()
        xAxisLen = xlim[1] - xlim[0]
        # dprint(xAxisLen)
        self.xAxisLenLabel.setText(f"xAxisLen: {xAxisLen:.2f}")

class PlotWindow(QMainWindow):
    def __init__(self, df, df_ax2=None):
        super().__init__()
        # Flag to check if it's the initial plot
        self.initial_plot = True

        self.setWindowTitle("Interactive Plot")

        # Store the DataFrame
        self.df = df
        self.df_ax2 = pd.DataFrame() if df_ax2 is None else df_ax2

        # Create central widget
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        layout = QVBoxLayout(self.central_widget)

        # Create Figure and Canvas
        self.fig = Figure()
        self.canvas = FigureCanvas(self.fig)
        self.toolbar = CustomNavigationToolbar(self.canvas, self, plot_window=self)
        
        self.fig.clear()
        self.ax1 = self.fig.add_subplot(111)
        # Set y-axis label
        self.ax1.set_ylabel('Flöde [m³/h]')

        # Create secondary axis if df_ax2 is provided
        if not self.df_ax2.empty:
            self.ax2 = self.ax1.twinx()
            self.ax2.set_ylabel('Kumsum volym [m³]')
        else:
            self.ax2 = None

        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        # Create buttons dynamically
        self.series_visible = [True] * (len(df.columns) + len(self.df_ax2.columns))
        checkbox_layout = QHBoxLayout()
        self.checkboxes = []
        
        # Add checkboxes for primary axis series
        for i, col in enumerate(df.columns):
            checkbox = QCheckBox(f"{col}")
            checkbox.setChecked(True)
            checkbox.stateChanged.connect(self.create_toggle_function(i))
            checkbox_layout.addWidget(checkbox)
            self.checkboxes.append(checkbox)
        
        # Add checkboxes for secondary axis series
        if not self.df_ax2.empty:
            for i, col in enumerate(self.df_ax2.columns, start=len(df.columns)):
                checkbox = QCheckBox(f"{col}")
                checkbox.setChecked(True)
                checkbox.stateChanged.connect(self.create_toggle_function(i))
                checkbox_layout.addWidget(checkbox)
                self.checkboxes.append(checkbox)
        
        layout.addLayout(checkbox_layout)

        # Initial plot
        self.plot()

    def create_toggle_function(self, index):
        def toggle():
            self.series_visible[index] = not self.series_visible[index]
            self.plot()
        return toggle

    def plot(self):
        # Save current axis limits if not initial plot
        if not self.initial_plot:
            xlim = self.ax1.get_xlim()
            ylim = self.ax1.get_ylim()
            if self.ax2:
                ylim2 = self.ax2.get_ylim()

        # Clear the figure and recreate axes
        self.fig.clear()
        self.ax1 = self.fig.add_subplot(111)
        if not self.df_ax2.empty:
            self.ax2 = self.ax1.twinx()
            self.ax2.set_ylabel('Kumsum volym [m³]')

        # Set up primary axis
        self.ax1.set_ylabel('Flöde [m³/h]')
        self.ax1.grid(visible=True, which='major', axis='both', color='grey')
        self.ax1.grid(visible=True, which='minor', axis='both', color='lightgrey')
        self.ax1.tick_params(which='minor', labelcolor='lightgrey')
        self.ax1.tick_params(axis='x', rotation=90, which='both')

        # Plot primary axis data
        handles, labels = [], []
        for i, col in enumerate(self.df.columns):
            if self.series_visible[i]:
                line, = self.ax1.plot(self.df.index, self.df[col], label=col, alpha=0.5)
                handles.append(line)
                labels.append(col)

        # Plot secondary axis data if it exists
        if not self.df_ax2.empty and self.ax2 is not None:
            for i, col in enumerate(self.df_ax2.columns, start=len(self.df.columns)):
                if self.series_visible[i]:
                    line, = self.ax2.plot(self.df.index, self.df_ax2[col], label=col, alpha=0.5)
                    handles.append(line)
                    labels.append(col)

        # Restore axis limits if not initial plot
        if not self.initial_plot:
            self.ax1.set_xlim(xlim)
            self.ax1.set_ylim(ylim)
            if self.ax2:
                self.ax2.set_ylim(ylim2)
        else:
            self.initial_plot = False
            self.ax1.autoscale(axis='both')
            if self.ax2:
                self.ax2.autoscale(axis='y')

        # Add legend
        self.ax1.legend(handles, labels)
        self.fig.tight_layout()
        self.canvas.draw()

    def resetXAxis(self):
        ax1 = self.canvas.figure.gca()
        xlim = ax1.get_xlim()
        xAxisLen = xlim[1]-xlim[0]

        if xAxisLen < 3/24:
            ax1.xaxis.set_major_locator(mdates.HourLocator(byhour=range(24),interval=1))
            ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(60),interval=1))
            ax1.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
            ax1.xaxis.set_minor_formatter(mdates.DateFormatter(r'%H:%M'))
        elif xAxisLen < 6/24:
            ax1.xaxis.set_major_locator(mdates.HourLocator(byhour=range(24),interval=1))
            ax1.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(60),interval=15))
            ax1.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
            ax1.xaxis.set_minor_formatter(mdates.DateFormatter(r'%H:%M'))
        elif xAxisLen < 2:
            ax1.xaxis.set_major_locator(mdates.DayLocator(bymonthday=range(1, 32),interval=1))
            ax1.xaxis.set_minor_locator(mdates.HourLocator(byhour=range(24),interval=1))
            ax1.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
            ax1.xaxis.set_minor_formatter(mdates.DateFormatter(r'%H:%M'))
        elif xAxisLen < 7:
            ax1.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=MO,interval=1))
            ax1.xaxis.set_minor_locator(mdates.HourLocator(interval=1))
            ax1.xaxis.set_major_formatter(mdates.DateFormatter(r'W%U-%m-%d %H:%M'))
            ax1.xaxis.set_minor_formatter(mdates.DateFormatter(r'%Y-%m-%d %H:%M'))
        elif xAxisLen <60:
            ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1,interval=1))
            ax1.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=MO,interval=1))
            ax1.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y-%m-%d'))
            ax1.xaxis.set_minor_formatter(mdates.DateFormatter(r'W%U-%m-%d'))
        else:
            ax1.xaxis.set_major_locator(mdates.YearLocator(interval=1))
            ax1.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=1,interval=1))
            ax1.xaxis.set_major_formatter(mdates.DateFormatter(r'%Y'))
            ax1.xaxis.set_minor_formatter(mdates.DateFormatter(r'%Y-%m-%d'))
        self.canvas.draw()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainWin = PlotWindow(df, df_ax2)
    mainWin.show()
    sys.exit(app.exec())

SystemExit: 0

c:\Users\chrini\sourceGit\chrisgladious\InteractiveFlowCharts\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
